In [42]:
import pandas as pd
# Importing the dataset
dataset = pd.read_csv('churn_modelling.csv')
X = dataset.iloc[:, 3:13]
y = dataset.iloc[:, 13]
#Create dummy variables
dataset.head()


,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [43]:
dataset.isnull().sum()

RowNumber          0
CustomerId         0
Surname            0
CreditScore        0
Geography          0
Gender             0
Age                0
Tenure             0
Balance            0
NumOfProducts      0
HasCrCard          0
IsActiveMember     0
EstimatedSalary    0
Exited             0
dtype: int64

In [44]:
X.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,619,France,Female,42,2,0.00,1,1,1,101348.88
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58
2,502,France,Female,42,8,159660.80,3,1,0,113931.57
3,699,France,Female,39,1,0.00,2,0,0,93826.63
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10


In [45]:
y.head()

0    1
1    0
2    1
3    0
4    0
Name: Exited, dtype: int64

In [47]:
geography=pd.get_dummies(X["Geography"],drop_first=True)
gender=pd.get_dummies(X['Gender'],drop_first=True)

In [14]:
geography

,Germany,Spain
0,0,0
1,0,1
2,0,0
3,0,0
4,0,1
...,...,...
9995,0,0
9996,0,0
9997,0,0
9998,1,0


In [48]:

X=pd.concat([X,geography,gender],axis=1)

## Drop Unnecessary columns
X=X.drop(['Geography','Gender'],axis=1)

In [32]:
X.shape

(10000, 14)

In [33]:
X.head()

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Germany,Spain,Male,Germany,Spain,Male
0,619,42,2,0.00,1,1,1,101348.88,0,0,0,0,0,0
1,608,41,1,83807.86,1,0,1,112542.58,0,1,0,0,1,0
2,502,42,8,159660.80,3,1,0,113931.57,0,0,0,0,0,0
3,699,39,1,0.00,2,0,0,93826.63,0,0,0,0,0,0
4,850,43,2,125510.82,1,1,1,79084.10,0,1,0,0,1,0


In [49]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

In [50]:
# Feature Scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [51]:
def build_model(hp):
    model = keras.Sequential()
    for i in range(hp.Int('num_layers', 2, 20)):
        model.add(layers.Dense(units=hp.Int('units_' + str(i),
                                            min_value=32,
                                            max_value=512,
                                            step=32),
                               activation='relu'))
    model.add(layers.Dense(1, activation='sigmoid'))
    model.compile(
        optimizer=keras.optimizers.Adam(
            hp.Choice('learning_rate', [1e-2, 1e-3, 1e-4])),
        loss='binary_crossentropy',
        metrics=['accuracy'])
    return model

In [52]:
from tensorflow import keras
from tensorflow.keras import layers
from kerastuner.tuners import RandomSearch
tuner = RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=3,
    executions_per_trial=3,
    directory='Weights',
    project_name='Churn')

In [53]:
tuner.search_space_summary()

In [54]:
tuner.search(X_train, y_train,
             epochs=5,
             validation_data=(X_test, y_test))

Epoch 1/5
250/250 [==============================] - ETA: 0s - loss: 0.6913 - accuracy: 0.78 - ETA: 0s - loss: 0.5057 - accuracy: 0.79 - ETA: 0s - loss: 0.4987 - accuracy: 0.78 - ETA: 0s - loss: 0.4854 - accuracy: 0.79 - ETA: 0s - loss: 0.4691 - accuracy: 0.79 - ETA: 0s - loss: 0.4578 - accuracy: 0.80 - 1s 2ms/step - loss: 0.4520 - accuracy: 0.8062 - val_loss: 0.3892 - val_accuracy: 0.8440
Epoch 2/5
250/250 [==============================] - ETA: 0s - loss: 0.3219 - accuracy: 0.81 - ETA: 0s - loss: 0.3823 - accuracy: 0.83 - ETA: 0s - loss: 0.3986 - accuracy: 0.82 - ETA: 0s - loss: 0.4049 - accuracy: 0.82 - ETA: 0s - loss: 0.4166 - accuracy: 0.81 - ETA: 0s - loss: 0.4100 - accuracy: 0.81 - ETA: 0s - loss: 0.4126 - accuracy: 0.81 - 0s 1ms/step - loss: 0.4111 - accuracy: 0.8186 - val_loss: 0.4125 - val_accuracy: 0.8090
Epoch 3/5
250/250 [==============================] - ETA: 0s - loss: 0.3789 - accuracy: 0.84 - ETA: 0s - loss: 0.3864 - accuracy: 0.83 - ETA: 0s - loss: 0.3914 - accuracy: 

Epoch 1/5
250/250 [==============================] - ETA: 0s - loss: 0.6909 - accuracy: 0.84 - ETA: 0s - loss: 0.5073 - accuracy: 0.81 - ETA: 0s - loss: 0.5077 - accuracy: 0.78 - ETA: 0s - loss: 0.4731 - accuracy: 0.79 - ETA: 0s - loss: 0.4612 - accuracy: 0.79 - ETA: 0s - loss: 0.4615 - accuracy: 0.79 - ETA: 0s - loss: 0.4506 - accuracy: 0.80 - ETA: 0s - loss: 0.4412 - accuracy: 0.81 - ETA: 0s - loss: 0.4348 - accuracy: 0.81 - ETA: 0s - loss: 0.4340 - accuracy: 0.81 - ETA: 0s - loss: 0.4299 - accuracy: 0.81 - ETA: 0s - loss: 0.4243 - accuracy: 0.82 - ETA: 0s - loss: 0.4193 - accuracy: 0.82 - ETA: 0s - loss: 0.4162 - accuracy: 0.82 - 1s 4ms/step - loss: 0.4162 - accuracy: 0.8254 - val_loss: 0.3737 - val_accuracy: 0.8470
Epoch 2/5
250/250 [==============================] - ETA: 0s - loss: 0.2694 - accuracy: 0.90 - ETA: 0s - loss: 0.4089 - accuracy: 0.82 - ETA: 0s - loss: 0.3876 - accuracy: 0.83 - ETA: 0s - loss: 0.3760 - accuracy: 0.83 - ETA: 0s - loss: 0.3859 - accuracy: 0.83 - ETA: 0s 

250/250 [==============================] - ETA: 0s - loss: 0.2602 - accuracy: 0.93 - ETA: 0s - loss: 0.3289 - accuracy: 0.87 - ETA: 0s - loss: 0.3567 - accuracy: 0.86 - ETA: 0s - loss: 0.3642 - accuracy: 0.84 - ETA: 0s - loss: 0.3684 - accuracy: 0.84 - ETA: 0s - loss: 0.3788 - accuracy: 0.84 - ETA: 0s - loss: 0.3726 - accuracy: 0.84 - ETA: 0s - loss: 0.3663 - accuracy: 0.85 - ETA: 0s - loss: 0.3695 - accuracy: 0.84 - ETA: 0s - loss: 0.3709 - accuracy: 0.84 - ETA: 0s - loss: 0.3733 - accuracy: 0.84 - ETA: 0s - loss: 0.3719 - accuracy: 0.84 - ETA: 0s - loss: 0.3704 - accuracy: 0.84 - 1s 3ms/step - loss: 0.3698 - accuracy: 0.8490 - val_loss: 0.3510 - val_accuracy: 0.8575
Epoch 3/5
250/250 [==============================] - ETA: 0s - loss: 0.4981 - accuracy: 0.71 - ETA: 0s - loss: 0.3449 - accuracy: 0.86 - ETA: 0s - loss: 0.3679 - accuracy: 0.85 - ETA: 0s - loss: 0.3384 - accuracy: 0.86 - ETA: 0s - loss: 0.3350 - accuracy: 0.86 - ETA: 0s - loss: 0.3359 - accuracy: 0.86 - ETA: 0s - loss: 0.

Epoch 1/5
250/250 [==============================] - ETA: 0s - loss: 0.6927 - accuracy: 0.71 - ETA: 1s - loss: 0.5764 - accuracy: 0.77 - ETA: 1s - loss: 0.5460 - accuracy: 0.77 - ETA: 1s - loss: 0.5268 - accuracy: 0.77 - ETA: 1s - loss: 0.5144 - accuracy: 0.77 - ETA: 1s - loss: 0.4899 - accuracy: 0.77 - ETA: 1s - loss: 0.4954 - accuracy: 0.77 - ETA: 1s - loss: 0.4795 - accuracy: 0.77 - ETA: 1s - loss: 0.4724 - accuracy: 0.78 - ETA: 0s - loss: 0.4675 - accuracy: 0.78 - ETA: 0s - loss: 0.4626 - accuracy: 0.78 - ETA: 0s - loss: 0.4582 - accuracy: 0.78 - ETA: 0s - loss: 0.4515 - accuracy: 0.79 - ETA: 0s - loss: 0.4495 - accuracy: 0.80 - ETA: 0s - loss: 0.4460 - accuracy: 0.80 - ETA: 0s - loss: 0.4408 - accuracy: 0.80 - ETA: 0s - loss: 0.4398 - accuracy: 0.80 - ETA: 0s - loss: 0.4385 - accuracy: 0.80 - ETA: 0s - loss: 0.4379 - accuracy: 0.80 - ETA: 0s - loss: 0.4348 - accuracy: 0.81 - ETA: 0s - loss: 0.4321 - accuracy: 0.81 - ETA: 0s - loss: 0.4284 - accuracy: 0.81 - ETA: 0s - loss: 0.4281 

250/250 [==============================] - ETA: 0s - loss: 0.5233 - accuracy: 0.68 - ETA: 1s - loss: 0.3945 - accuracy: 0.82 - ETA: 1s - loss: 0.3623 - accuracy: 0.84 - ETA: 1s - loss: 0.3686 - accuracy: 0.84 - ETA: 1s - loss: 0.3818 - accuracy: 0.83 - ETA: 1s - loss: 0.3810 - accuracy: 0.83 - ETA: 0s - loss: 0.3737 - accuracy: 0.83 - ETA: 0s - loss: 0.3686 - accuracy: 0.84 - ETA: 0s - loss: 0.3698 - accuracy: 0.84 - ETA: 0s - loss: 0.3725 - accuracy: 0.84 - ETA: 0s - loss: 0.3729 - accuracy: 0.84 - ETA: 0s - loss: 0.3735 - accuracy: 0.84 - ETA: 0s - loss: 0.3703 - accuracy: 0.84 - ETA: 0s - loss: 0.3765 - accuracy: 0.84 - ETA: 0s - loss: 0.3766 - accuracy: 0.83 - ETA: 0s - loss: 0.3772 - accuracy: 0.84 - ETA: 0s - loss: 0.3804 - accuracy: 0.83 - ETA: 0s - loss: 0.3763 - accuracy: 0.84 - ETA: 0s - loss: 0.3764 - accuracy: 0.84 - ETA: 0s - loss: 0.3747 - accuracy: 0.84 - ETA: 0s - loss: 0.3737 - accuracy: 0.84 - ETA: 0s - loss: 0.3721 - accuracy: 0.84 - ETA: 0s - loss: 0.3714 - accuracy

250/250 [==============================] - ETA: 0s - loss: 0.2581 - accuracy: 0.87 - ETA: 1s - loss: 0.3230 - accuracy: 0.87 - ETA: 1s - loss: 0.3452 - accuracy: 0.86 - ETA: 1s - loss: 0.3389 - accuracy: 0.86 - ETA: 1s - loss: 0.3296 - accuracy: 0.87 - ETA: 1s - loss: 0.3340 - accuracy: 0.86 - ETA: 0s - loss: 0.3365 - accuracy: 0.86 - ETA: 0s - loss: 0.3468 - accuracy: 0.85 - ETA: 0s - loss: 0.3487 - accuracy: 0.85 - ETA: 0s - loss: 0.3536 - accuracy: 0.85 - ETA: 0s - loss: 0.3495 - accuracy: 0.85 - ETA: 0s - loss: 0.3494 - accuracy: 0.85 - ETA: 0s - loss: 0.3446 - accuracy: 0.86 - ETA: 0s - loss: 0.3473 - accuracy: 0.86 - ETA: 0s - loss: 0.3459 - accuracy: 0.86 - ETA: 0s - loss: 0.3432 - accuracy: 0.86 - ETA: 0s - loss: 0.3462 - accuracy: 0.86 - ETA: 0s - loss: 0.3475 - accuracy: 0.85 - ETA: 0s - loss: 0.3494 - accuracy: 0.85 - ETA: 0s - loss: 0.3479 - accuracy: 0.85 - ETA: 0s - loss: 0.3457 - accuracy: 0.85 - ETA: 0s - loss: 0.3447 - accuracy: 0.86 - ETA: 0s - loss: 0.3433 - accuracy

INFO:tensorflow:Oracle triggered exit


In [55]:
tuner.get_best_hyperparameters

<bound method BaseTuner.get_best_hyperparameters of <kerastuner.tuners.randomsearch.RandomSearch object at 0x0000017047D7EB80>>

In [56]:
tuner.get_best_models()